In [1]:
from os.path import abspath
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

spark = SparkSession \
    .builder \
    .appName("renovation") \
    .getOrCreate()

# set this parameter for date issue before 1582 (dpe database)
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [5]:
file_location = "../../data/dpe_france_2021.csv"
file_type = "csv"
name = "dpe_2021"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)
if name == "weather":
    new_column_name_list= [name.replace(',','') for name in df.columns]
    df = df.toDF(*new_column_name_list)

In [6]:
df.show()

+-------------+------------------+----------------------+--------------------------+--------------------+---------------+---------------------+-----------+-----------------------+----------------------------+-----------------------+------------------------------+-------------------------+----------------+-------------+-------------+------------------+-------------+---------------------------+-------------------------------+--------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+-----------------------+------------------+-------------------------------+----------------+----------------+--------------------+------------------+----------------+-------------+----------------+--------------------+-----------------+---------+--------------------+---------------------------------+---------------------------------+------------------+--------------------+----------

In [26]:
energy_list = [
    "Électricité",
	"Gaz naturel",
	"Réseau de Chauffage urbain",
	"Fioul domestique",
	"Bois – Bûches",
	"Bois – Granulés (pellets) ou briquettes",
	"GPL",
	"Propane",
	"Bois – Plaquettes forestières",
	"Charbon",
	"Bois – Plaquettes d’industrie",
	"Électricité d'origine renouvelable utilisée dans le bâtiment",
	"Butane",
	"Réseau de Froid Urbain"
]

dpe_2021 = (
    df.select(
        "N°DPE",
        "Date_établissement_DPE",
        "Année_construction",
        "Type_bâtiment",
        "Type_installation_chauffage",
        "Type_installation_ECS_(général)",
        "Surface_habitable_logement",
        "Code_INSEE_(BAN)",
        "Code_postal_(BAN)",
        "Conso_5_usages_é_finale",
        "Emission_GES_5_usages",
        "Type_installation_ECS",
        "Type_énergie_principale_chauffage",
        "Type_émetteur_installation_chauffage_n°1",
        "Type_générateur_n°1_installation_n°1",
        "Qualité_isolation_enveloppe",
        "Qualité_isolation_menuiseries",
        "Qualité_isolation_murs",
        "Qualité_isolation_plancher_bas"
    )
    .filter(
        # not null values
        (F.col("Surface_habitable_logement").isNotNull()) &
        (F.col("Code_postal_(BAN)").isNotNull()) &
        (F.col("Code_INSEE_(BAN)").isNotNull()) &
        (F.col("Conso_5_usages_é_finale").isNotNull()) &
        (F.col("Emission_GES_5_usages").isNotNull()) &
        (F.col("Type_installation_ECS").isNotNull()) &
        (F.col("Type_énergie_principale_chauffage").isNotNull()) &
        (F.col("Type_émetteur_installation_chauffage_n°1").isNotNull()) &
        (F.col("Type_générateur_n°1_installation_n°1").isNotNull()) &

        # not an irrelevant value
        (F.col("Type_énergie_principale_chauffage").isin(energy_list)) &
        (F.col("Type_installation_chauffage_n°1").cast("float").isNull()) &
        (F.col("Type_énergie_générateur_ECS_n°1").isin(energy_list)) &
        (F.col("Type_bâtiment") != "immeuble")
    )
)

print(f'{df.count() = }, {dpe_2021.count() = }')

df.count() = 4277643, dpe_2021.count() = 3779166


In [7]:
# dpe_2021_filter.dtypes
dpe_2021.show()

+-------------+----------------------+--------------------+---------------+-----------+-----------------------+-----------------------+-------------+-------------+------------------+-------------+---------------------------+-------------------------------+--------------------+------------------+----------------------+----------------------+--------------------------+-------------------------------+----------------+----------------+--------------------+-----------------+--------------------+-----------------------+--------------------------+---------------------+----------------------------+---------------------+----------------------------+-------------------------------+-----------------------+---------------------------------+-------------------------------+----------------------------------------+-----------------------------------------------------+--------------------------------------------+------------------------------------+-----------+-----------------------------------------

In [10]:
def show_distinct(df, variable, treshold=10):
    return_value = (
        df.select(
            F.col(variable)
        )
        .groupBy(
            F.col(variable)
        )
        .count()
        .filter(F.col('count') > treshold)
        .orderBy(F.desc(F.col('count')))
        .collect()
    )
    print(f'\n {variable:_^100} : \n')
    
    for value in return_value:
        print(f'{value[variable]}, count : {value["count"]}')
    return return_value

## Chauffage

In [9]:
show_distinct(dpe_2021, 'Type_énergie_principale_chauffage')
show_distinct(dpe_2021, 'Type_installation_chauffage_n°1')
show_distinct(dpe_2021, 'Type_émetteur_installation_chauffage_n°1')
show_distinct(dpe_2021, 'Description_générateur_chauffage_n°1_installation_n°1')
show_distinct(dpe_2021, 'Type_énergie_générateur_n°1_installation_n°1')
show_distinct(dpe_2021, 'Type_générateur_n°1_installation_n°1')
print('')



 _________________________________Type_énergie_principale_chauffage__________________________________ : 

Électricité, count : 1629171
Gaz naturel, count : 1495423
Réseau de Chauffage urbain, count : 337061
Fioul domestique, count : 212223
Bois – Bûches, count : 93973
Bois – Granulés (pellets) ou briquettes, count : 32879
GPL, count : 31581
Propane, count : 7355
Bois – Plaquettes forestières, count : 1358
Charbon, count : 1031
Bois – Plaquettes d’industrie, count : 742
Électricité d'origine renouvelable utilisée dans le bâtiment, count : 305
Butane, count : 205
Réseau de Froid Urbain, count : 23

 __________________________________Type_installation_chauffage_n°1___________________________________ : 

installation individuelle, count : 2926984
installation collective, count : 752307
installation collective multi-bâtiment : modélisée comme un réseau de chaleur, count : 140982
installation hybride collective-individuelle (chauffage base + appoint individuel ou convecteur bi-jonction), co

## ECS

In [10]:
show_distinct(dpe_2021, 'Type_installation_ECS')
show_distinct(dpe_2021, 'Type_énergie_générateur_ECS_n°1')
show_distinct(dpe_2021, 'Type_générateur_ECS_n°1')
show_distinct(dpe_2021, 'Description_installation_ECS')
print('')


 _______________________________________Type_installation_ECS________________________________________ : 

installation individuelle, count : 3175894
installation collective, count : 614920
installation collective multi-bâtiment : modélisée comme un réseau de chaleur, count : 52516

 __________________________________Type_énergie_générateur_ECS_n°1___________________________________ : 

Électricité, count : 2024305
Gaz naturel, count : 1422989
Réseau de Chauffage urbain, count : 234463
Fioul domestique, count : 114343
GPL, count : 29095
Propane, count : 6550
Bois – Granulés (pellets) ou briquettes, count : 5437
Bois – Bûches, count : 4179
Butane, count : 784
Bois – Plaquettes forestières, count : 648
Électricité d'origine renouvelable utilisée dans le bâtiment, count : 270
Bois – Plaquettes d’industrie, count : 231
Charbon, count : 30

 ______________________________________Type_générateur_ECS_n°1_______________________________________ : 

Ballon électrique à accumulation vertical Caté

In [11]:
dpe_2021.filter(
    (F.col("Type_bâtiment") == 'appartement') &
    (F.col("Nombre_niveau_logement") >= 1)
).select(
        F.col("Surface_habitable_logement"),
        F.col("Nombre_appartement"),
        F.col("Nombre_niveau_immeuble"),
        F.col("Nombre_niveau_logement"),
        F.col("Surface_habitable_logement"),
        F.col("Position_logement_dans_immeuble")
).groupBy(F.col("Nombre_niveau_logement")).agg(F.avg(F.col('Surface_habitable_logement'))).orderBy('Nombre_niveau_logement').show()


+----------------------+-------------------------------+
|Nombre_niveau_logement|avg(Surface_habitable_logement)|
+----------------------+-------------------------------+
|                     1|              55.52459416631139|
|                     2|              71.28600598965423|
|                     3|              67.15659266349014|
|                     4|               61.4982553012002|
|                     5|              62.55774393683023|
|                     6|              58.09521065068296|
|                     7|              65.42083050719722|
|                     8|              65.47071988064155|
|                     9|              68.38417369459896|
|                    10|             63.098926829268294|
|                    11|               66.6309127248501|
|                    12|              63.41518275538894|
|                    13|              77.45403899721448|
|                    14|              88.33583489681051|
|                    15|       

In [12]:
show_distinct(dpe_2021.filter(F.col('Description_installation_ECS').contains('thermodynamique')), 'Type_énergie_générateur_ECS_n°1')



 __________________________________Type_énergie_générateur_ECS_n°1___________________________________ : 

Électricité, count : 55425
Électricité d'origine renouvelable utilisée dans le bâtiment, count : 265


[Row(Type_énergie_générateur_ECS_n°1='Électricité', count=55425),
 Row(Type_énergie_générateur_ECS_n°1="Électricité d'origine renouvelable utilisée dans le bâtiment", count=265)]

In [18]:
def cut_string(string, lenght):
    if not string:
        return 'None'
    if len(str(string)) > lenght:
        return string[0:lenght]
    else:
        return string

def print_df(collection, variables, length):
    print(''.join([f"{variable:<{length}}" for variable in variables]))
    for value in collection:
        print(''.join([f'{cut_string(value[variable], length):<{length}}' for variable in variables]))



def see_combined(df, condition, variables, length):
    if condition : 
        study = df.filter(condition)
    else: 
        study = df
    collection = (
        study.select([col for col in variables])
        .groupBy([col for col in variables])
        .count()
        .orderBy(F.desc('count'))
        .collect()
    )
    print_df(collection, variables + ['count'], length)
    return collection
    
    



In [14]:
see_combined(dpe_2021, 'Description_installation_ECS = "Combiné au système de chauffage"', ['Type_énergie_principale_chauffage', 'Type_énergie_générateur_ECS_n°1'], 50)

Type_énergie_principale_chauffage                 Type_énergie_générateur_ECS_n°1                   count                                             
Gaz naturel                                       Gaz naturel                                       721363                                            
Réseau de Chauffage urbain                        Réseau de Chauffage urbain                        120711                                            
Fioul domestique                                  Fioul domestique                                  45549                                             
GPL                                               GPL                                               12949                                             
Électricité                                       Électricité                                       10825                                             
Électricité                                       Gaz naturel                                 